In [27]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def stockReturns(priceDF):
    
    # Method 1: Compute daily returns by parsing the Data Frame as numpy matrix (Fastest)
    compTickers = priceDF.columns[1: ]    
    priceMat = priceDF.loc[ : , compTickers].as_matrix()    
    diffMat = (priceMat[1: ] - priceMat[ :-1]) / priceMat[ :-1]
    
    return pd.DataFrame(data = diffMat, index = priceDF.index[1: ], columns = compTickers)

def calCorrelations(dailyReturn):
    
    # Method 1: Pandas built-in function to calculate pairwise correlation (Fastest)
#    return dailyReturn.corr()
    
    # Method 2: Manual calculation of pairwise correlation using numpy matrix (Faster)
    col = dailyReturn.columns
    ncol = len(col)
    corrMat = np.identity(ncol)
    
    G = nx.Graph()
    G.add_nodes_from(col.values)
        
    n = len(dailyReturn)
    for i in range(0, ncol):
        for j in range(i + 1, ncol):
            x = dailyReturn[col[i]]
            y = dailyReturn[col[j]]
            xsum = sum(x)
            ysum = sum(y)
            corrMat[i][j] = (n * sum(x * y) - xsum * ysum) / (np.sqrt(n * sum(x**2) - xsum**2) * np.sqrt(n * sum(y**2) - ysum**2))
            corrMat[j][i] = corrMat[i][j]
            G.add_edge(col[i], col[j], weight = corrMat[i][j])
        
    return pd.DataFrame(data = corrMat, index = col, columns = col), G

def stockClustering(graph, k):
            
    # Method 1: Clustering using dictionary and list
    sortedEdges = sorted(graph.edges(data = True), key = lambda edge: edge[2]['weight'], reverse = True) # O(e log e), e = number of edges = n(n - 1)/2 for dense graph
    
    listSets = dict()    
    for node in graph.nodes(): # O(n), n = number of companies
        listSets[node] = {node}
        
    for i in range(0, k): # Execute k times
        (a, b, data) = sortedEdges[i]
        mergedSet = listSets[a].union(listSets[b]) # O(len(set(a)) + len(set(b))) = O(n) in worst case since the biggest possible set is the set with all n companies
        for node in mergedSet: # O(n)
            listSets[node] = mergedSet
    # The whole chunk above is O(kn + kn) = O(kn)
            
    resultSets = []
    for nodeSet in listSets.values(): # O(n)
        if nodeSet not in resultSets:
            resultSets.append(nodeSet)
        
    return resultSets

priceDF = pd.read_csv('SP_500_close_2015.csv')
firmDF = pd.read_csv('SP_500_firms.csv', index_col = 0)

dailyReturn = stockReturns(priceDF)

corrDF, corrGraph = calCorrelations(dailyReturn)

cluster = stockClustering(corrGraph, 1000)

def getStockDetails(ticker, clusterSets, firmDF):
    
    # if the ticker given is in a single set cluster
    if {ticker} in clusterSets:
        return firmDF.loc[ticker, :]
    else:
        for i in range(len(clusterSets)):
            if ticker in clusterSets[i]:
                return firmDF.loc[clusterSets[i], :].sort_values(['Sector', 'Name'])
            


## Average Linkage Clustering 

The first alternative clustering algorithm we will be looking at is Average Linkage Clustering. Both Average-Linkage Clustering and Single-Linkage Clustering are subtypes of Agglomerative Hierarchical Clustering. The difference between these two clustering methods is the definition of 'shortest distance'. Note that in our earlier example, correlation between stocks was used to represent similarities. We can think of correlation as the inverse of 'distance' between the nodes.  

In single-linkage clustering, the distance between two clusters is determined by a single element pair (one from each cluster) which are closest to each other. The algorithm always chooses the shortest of these links while merging two clusters. While merging cluster, other than the aforementioned element pair, single-linkage clustering disregards the distances to all other elements in the cluster. Consequently, it tends to produce large cluster where ceertain pairs of elements are dissimilar to each other.  

However, in average-linkage clustering, the distance between two clusters is defined as the average of all distances between pairs of objects (each from the two distinct clusters). This helps to prevent the so-called chaining phenomenon experienced by single-linkage clustering, where many of the elements in each cluster may be very dissimilar.  

### Python Code  

The following section includes the Python code in performing average-linkage clustering using scikit-learn. Since the scikit-learn function requires a distance matrix, we will need to convert the correlation matrix into distance matrix.    

Convert the correlation matrix into distance matrix. Correlation is within the interval [-1, 1] and a higher value reflects greater similarity. We'll need to transform correlation value into distance, hence we get the absolute value of (correlation - 1). The resulting value will be a distance within interval [0, 2] and a higher value reflects lower similarity.  


In [28]:
from sklearn.cluster import AgglomerativeClustering

def stockClusteringAgglomerative(corrDF, num_clusters):
    
    # Store column names as company's ticker symbol
    compTickers = corrDF.columns
    
    # Transform correlation matrix into distance matrix
    affinityMatrix = abs(corrDF.as_matrix() - 1)
    
    # Run agglomerative clustering with average-linkage with precomputed distance matrix
    model = AgglomerativeClustering(linkage = 'average', affinity = 'precomputed', n_clusters = num_clusters)
    aggFit = model.fit(affinityMatrix)
    
    resultSets = [set()] * num_clusters
    i = 0
    
    # Return the clustering results in the form of "list of sets", each set representing a cluster
    for l in aggFit.labels_:
        resultSets[l] = resultSets[l].union({compTickers[i]})
        i += 1
        
    return resultSets

## K-Means Clustering 

The second clustering algorithm we use is K-Means Clustering. To use K-Means clustering, we need to define the "features" of each node. Correlation only provides the distance between each node and hence is not sufficient for K-Means clustering. In this case, we define the features to be daily price change (in percentage) of each individual stock. This information is contained in the daily return matrix where each column contains the daily price change in terms of percentage.  

K-Means clustering attempts to fit K number of centroids to the dataset such that the total sum of distance from each node to its corresponding centroid is minimised. One characterisitc of K-Means clustering is that it tends to separate data into clusters of roughly equal size. In other words, it assumes the data distribution to be separable with K spherical clusters. This might not work well on certain data where this assumption does not hold true.  

Lastly, since K-Means clustering assumes the data to be separable using K spherical clusters of roughly equal size, the choice of K value is key. A bad choice of K can lead to bad results. We will explore this in later section.  

### Python Code  

The following section includes the Python code in performing K-Means clustering using scikit-learn. As mentioned above, K-Means clustering function requires a list of vectors defining the "features" of each stock. We will use the daily return for K-Means clustering algorithm.  

In [29]:
from sklearn.cluster import KMeans

def stockClusteringKMeans(dailyReturn, num_clusters):
    
    # Store column names as company's ticker symbol
    compTickers = dailyReturn.columns
    
    # Transpose the daily return matrix so that each row contains all the price changes of a particular stock
    dailyReturnArray = dailyReturn.as_matrix().transpose()
    
    # Run K-Means clustering with random centroid initialisation
    kmeans = KMeans(n_clusters = num_clusters, random_state = 0).fit(dailyReturnArray)
    
    resultSets = [set()] * num_clusters
    i = 0
    
    # Return the clustering results in the form of "list of sets", each set representing a cluster
    for l in kmeans.labels_:
        resultSets[l] = resultSets[l].union({compTickers[i]})
        i += 1
    
    return resultSets


## Clustering Results Comparison

### Single-Linkage Clustering (K = 1000, Number of Clusters = 327)

As seen in earlier section, the "greedy" clustering algorithm (a.k.a. single-linkage clustering) tends to produce a very large cluster with dissimilar elements within the same cluster when K is large. For instance, it groups the following stocks within the same cluster as Goldman Sachs (GS).  

In [30]:
getStockDetails('GS', cluster, firmDF)

,Name,Sector
Symbol,,
SNA,Snap-On Inc.,Consumer Discretionary
SWK,Stanley Black & Decker,Consumer Discretionary
AFL,AFLAC Inc,Financials
AMG,Affiliated Managers Group Inc,Financials
AIG,"American International Group, Inc.",Financials
AMP,Ameriprise Financial,Financials
AON,Aon plc,Financials
AJG,Arthur J. Gallagher & Co.,Financials
BBT,BB&T Corporation,Financials


We can notice that there are many non-financials stocks in this cluster. Many Consumer Discretionary, Health Care, Industrials and Information Technology stocks have been mis-classified under this cluster. This demonstrates the weakness of single-linkage clustering: it produces long thin clusters where nearby elements are similar, but elements at the opposite ends of a cluster may be very dissimilar.  

### Average-Linkage Clustering (Number of Clusters = 327)

Next, we will see how average linkage clustering addresses this issue. The average-linkage clustering is invoked to produce equally many clusters as the "greedy" single-linkage clustering algorithm.  

The below table tabulates the list of companies within the same cluster as Goldman Sachs if average-linkage clustering is used.  

In [31]:
# Call cluster-linkage clustering using correlation DF, and num_cluster = number of clusters produced earlier
clusterAggAveLinkage = stockClusteringAgglomerative(corrDF, len(cluster))

getStockDetails('GS', clusterAggAveLinkage, firmDF)

,Name,Sector
Symbol,,
AFL,AFLAC Inc,Financials
AMP,Ameriprise Financial,Financials
BBT,BB&T Corporation,Financials
BAC,Bank of America Corp,Financials
SCHW,Charles Schwab Corporation,Financials
C,Citigroup Inc.,Financials
CMA,Comerica Inc.,Financials
ETFC,E*Trade,Financials
FITB,Fifth Third Bancorp,Financials


As we can see, this cluster remains of manageable size and all the companies within this cluster are financials companies. All the elements are rather similar to each other, compared with the results obtained from single-linkage clustering.  

While computing the distance between two clusters, average-linkage clustering considers the average distance between all the element pairs from the two clusters and this has evidently produced a better clustering results than the "greedy" single-linkage clustering.  

### K-Means Clustering (K = 327)

Lastly, we shall inspect the clusters formed using K-Means clustering algorithm. We shall invoke the K-Means clustering algorithm to produce equally many clusters as the "greedy" single-linkage clustering algorithm.  

Let's find out which cluster Goldman Sachs belong to.  

In [32]:
# Call K-Means clustering using daily return DF, and num_cluster = number of clusters produced earlier
clusterKMeans = stockClusteringKMeans(dailyReturn, len(cluster))

getStockDetails('GS', clusterKMeans, firmDF)

,Name,Sector
Symbol,,
AMP,Ameriprise Financial,Financials
BAC,Bank of America Corp,Financials
C,Citigroup Inc.,Financials
GS,Goldman Sachs Group,Financials
JPM,JPMorgan Chase & Co.,Financials
MS,Morgan Stanley,Financials
STT,State Street Corp.,Financials
BK,The Bank of New York Mellon Corp.,Financials


As mentioned in earlier section, K-Means clustering tends to separate the data into clusters of equal size. A high value of K would separate the companies into many small clusters. As we can see from the above table, the cluster (which Goldman Sachs belong to) is a much smaller cluster with only 8 companies.  

Other financials companies are separated into different clusters, such as Zions Bancrop:

In [33]:
getStockDetails('ZION', clusterKMeans, firmDF)

,Name,Sector
Symbol,,
CMA,Comerica Inc.,Financials
FITB,Fifth Third Bancorp,Financials
KEY,KeyCorp,Financials
RF,Regions Financial Corp.,Financials
ZION,Zions Bancorp,Financials


And U.S. Bancorp:

In [34]:
getStockDetails('USB', clusterKMeans, firmDF)

,Name,Sector
Symbol,,
BBT,BB&T Corporation,Financials
HBAN,Huntington Bancshares,Financials
MTB,M&T Bank Corp.,Financials
NTRS,Northern Trust Corp.,Financials
PNC,PNC Financial Services,Financials
PBCT,People's United Financial,Financials
STI,SunTrust Banks,Financials
USB,U.S. Bancorp,Financials
WFC,Wells Fargo,Financials


In this run of the K-Means Clustering algorithm, K is a very high number, 327. As we can see, a high number of K performs very granular division and separates similar financials companies into different clusters. A smaller number of K might potentially yield better results.  

### K-Means Clustering Revisited (K = 50)  

Since a high K value may give us too many small and granular clusters, we shall now try a lower value of K. In this section, we will rerun the algorithm with K = 50 and find out what other companies are in the same cluster as Goldman Sachs.  

In [35]:
# Re-run K-Means clustering with num_cluster = 50
clusterKMeans = stockClusteringKMeans(dailyReturn, 50)

getStockDetails('GS', clusterKMeans, firmDF)

,Name,Sector
Symbol,,
AMG,Affiliated Managers Group Inc,Financials
AIG,"American International Group, Inc.",Financials
AMP,Ameriprise Financial,Financials
BBT,BB&T Corporation,Financials
BAC,Bank of America Corp,Financials
COF,Capital One Financial,Financials
SCHW,Charles Schwab Corporation,Financials
C,Citigroup Inc.,Financials
CFG,Citizens Financial Group,Financials


This time, the cluster is much laarger and contains the other relevant financials companies. By reducing K, K-Means clustering divides the companies into a bigger cluster of similar companies.  

However, recall that K-Means clustering expects all the clusters to be of similar size. Reducing K also result in an unnecessarily large cluster like below.  

In [36]:
getStockDetails('JNJ', clusterKMeans, firmDF)

,Name,Sector
Symbol,,
AAP,Advance Auto Parts,Consumer Discretionary
HOG,Harley-Davidson,Consumer Discretionary
CVS,CVS Health,Consumer Staples
MJN,Mead Johnson,Consumer Staples
WBA,Walgreens Boots Alliance,Consumer Staples
WFM,Whole Foods Market,Consumer Staples
AXP,American Express Co,Financials
ABT,Abbott Laboratories,Health Care
A,Agilent Technologies Inc,Health Care


Johnson & Johnson, a pharmaceutical company, is correctly placed under the Health Care cluster. However, some companies in other sectors (such as Consumer Discretionary, Consumer Staples and Industrials) are also placed in the same cluster. As we can see, a lower K value results in overly large clusters with dissimilar companies.  

In conclusion, the choice of K is crucial in producing an accurate clustering result. Some diagnostic checks can be performed on the data in order to determine the near-optimal choice. Also, the assumption of near-equal sized spherical cluster may not hold true for certain data distribution and it may be better to use other clustering algorithms in our case (in clustering different stocks within S&P 500).